# tkinter - Quick and Dirty GUIs with Python

### "But I don't know how to use the command line"

*- User*

## So what is tkinter anyway?

* Tk is a GUI library originally built for the Tcl language [tcl-lang.org](https://www.tcl-lang.org/)
* tkinter is a Python wrapper around the Tk GUI library
* tkinter has been in the Python standard library a long time
* Ttk is a newer set of themed widgets (accessible via the `tkinter.ttk` module)


## Installing

Even though tkinter is part of the Python standard library, your Linux distribution might not have the tkinter 

`sudo apt-get install python3-tk`

## Hello World

In [1]:
# Hello World
from tkinter import Tk
from tkinter.ttk import Frame, Label, Button

root = Tk()  # create a root window
root.title('Hello World') # Set the title of the window

# Frame is a rectangular area where compoents can be placed
frame = Frame(root, padding=10)  
frame.grid()  # use grid layout to position widgets inside frame

# Add a Label and Button
Label(frame, text="Hello World").grid(column=0, row=0)
Button(frame, text="Quit", command=root.destroy).grid(column=0, row=1)

root.mainloop()

## Input/Output

In [14]:
# Encapsulate the boilerplate
def build_root_and_frame(title):
    root = Tk()
    root.title(title)
    frame = Frame(root, padding=10)
    frame.grid()
    return root, frame

def grid(widget, row, column, **kwargs):
    widget.grid(row=row, column=column, **kwargs)

In [3]:
# Take some input: Hello NAME
from tkinter import StringVar
from tkinter.ttk import Entry

root, frame = build_root_and_frame('Hello NAME')

# This is known as a control value it's the value underlying the widget
# also comes in DoubleVar and IntVar flavors
name_in = StringVar()
name_out = StringVar()

def greet(*args):
    name_out.set(f"Hello {name_in.get()}")

grid(Label(frame, text='Name'), 0, 0)
grid(Entry(frame, width=20, textvariable=name_in), 0, 1)
grid(Button(frame, text='Greet', command=greet), 0, 2)

grid(Label(frame, textvariable=name_out), 1, 0)
grid(Button(frame, text='Quit', command=root.destroy), 2, 0)

root.mainloop()

## Canvas

In [4]:
# another factory function
def grid_label(parent, text, row, column):
    grid(Label(parent, text=text), row, column)

In [5]:
from tkinter import Canvas

root, frame = build_root_and_frame('Canvas')
canvas = Canvas(frame, height=100, width=100, background='white')

def draw():
    canvas.create_line(0, 50, 50, 0)
    canvas.create_arc(50, 0, 100, 50)
    canvas.create_oval(50, 50, 100, 100)
    canvas.create_rectangle(25, 75, 75, 25)

grid(canvas, 0, 0)
grid(Button(frame, text='draw', command=draw), 0, 1)

root.mainloop()

## Checkbutton

In [6]:
from tkinter.ttk import Checkbutton

root, frame = build_root_and_frame('Checkbutton')

hackme = StringVar()
grid_label(frame, 'Checkbutton', 0, 0)
grid(
    Checkbutton(
        frame,
        text="Save credit card so it can be stolen later",
        variable=hackme,
        onvalue='hackme',
        offvalue="oh no you don't",
    ), 
    0, 1,
)
grid(Label(frame, textvariable=hackme), 1, 0)

root.mainloop()

## Radiobutton

In [7]:
from tkinter.ttk import Radiobutton

root, frame = build_root_and_frame('Radiobutton')

language = StringVar()
grid(Radiobutton(frame, text="English", variable=language, value='EN'), 0, 0)
grid(Radiobutton(frame, text="Españole", variable=language, value='ES'), 1, 0)
grid_label(frame, 'Value', 2, 0)
grid(Label(frame, textvariable=language), 2, 1)

root.mainloop()

## Radiobutton, with Style

The diamond Radiobuttons are wierd, but you can switch your app to a different theme to avoid them

In [19]:
from tkinter.ttk import Style

style = Style()
print(f"{style.theme_names()=}")
print(f"{style.theme_use()=}")

style.theme_names()=('clam', 'alt', 'default', 'classic')
style.theme_use()='default'


In [20]:
root = Tk()
root.style = Style(root)
root.style.theme_use('alt')
root.title("Radiobutton, with style")

frame = Frame(root, padding=10)
frame.grid()

language = StringVar()
grid(Radiobutton(frame, text="English", variable=language, value='EN'), 0, 0, sticky='w')
grid(Radiobutton(frame, text="Español", variable=language, value='ES'), 1, 0, sticky='w')
grid_label(frame, 'Value', 2, 0)
grid(Label(frame, textvariable=language), 2, 1)

root.mainloop()

In [23]:
from tkinter import Listbox

root, frame = build_root_and_frame('Listbox')

choices = ['apple', 'bananna', 'cucumber']
fruitchoice = StringVar(value=choices)
grid(Listbox(frame, listvariable=fruitchoice), 0, 0)
grid(Label(frame, textvariable=fruitchoice), 1, 0)

root.mainloop()

## References

* [TkDocs Tutorial](https://tkdocs.com/tutorial/index.html)
* [Tkinter Reference](https://tkdocs.com/shipman/)